# Chopper Cascades

## Overview

For the purpose of [frame unwrapping](frame-unwrapping.ipynb) as well as techniques such as wavelength-frame multiplication (WFM), we need to compute bounds of neutron arrival times.
This can be computed from the source pulse structure, the chopper configuration, and the instrument geometry.

The [chopper_cascade module](https://scipp.github.io/scippneutron/generated/modules/scippneutron.tof.chopper_cascade.html) provides utilities for this purpose.
It is currently under development and not fully functional.

## Example: WFM chopper cascade

As an example, consider the WFM chopper cascade from [the tof package documentation](https://tof.readthedocs.io/en/stable/short-example.html).
The choppers are defined in the `fakes` module of the `tof` package:

In [ ]:
from scippneutron.tof import chopper_cascade, fakes
import scipp as sc
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 300
choppers = fakes.wfm_choppers

We can now initialize a frame-sequence and apply the chopper cascade to it:

In [ ]:
frames = chopper_cascade.FrameSequence.from_source_pulse(
    time_min=sc.scalar(0.0, unit='ms'),
    time_max=sc.scalar(4.0, unit='ms'),  # ESS pulse is 3 ms, but it has a tail
    wavelength_min=sc.scalar(0.0, unit='angstrom'),
    wavelength_max=sc.scalar(10.0, unit='angstrom'),
)
frames = frames.chop(choppers.values())
at_sample = frames.propagate_to(sc.scalar(26.0, unit='m'))

at_sample.draw()

We can also draw a chopper acceptance diagram, which is essentially the same as above, but propagated back to the source pulse distance:

In [ ]:
frames.acceptance_diagram()

## Frame unwrapping

For unwrapping frames, we need the bounds of the entire from, to determine times at which to cut.
Since $L_2$ can be different for every detector, this cutting time is different for every detector.
We can compute the frame bounds at a common distance, e.g., the sample, and propagate the bounds to the detectors:

In [ ]:
bounds = at_sample[-1].bounds()
chopper_cascade.propagate_times(
    time=bounds['time'],
    wavelength=bounds['wavelength'],
    distance=sc.linspace('L2', 1.0, 2.0, 100, unit='m'),
)

For WFM, we need to compute subframe time cutting points.
Again, $L_2$ can be different for every detector, so we need to compute the cutting points for every detector:

In [ ]:
bounds = at_sample[-1].subbounds()
bounds

In [ ]:
chopper_cascade.propagate_times(
    time=bounds['time'],
    wavelength=bounds['wavelength'],
    distance=sc.linspace('L2', 1.0, 2.0, 100, unit='m'),
)

Note that subframes may in principle overlap, and this may depend on the detector.
We therefore call `subframe_bounds` at a common location and propagate the result, otherwise we would get a different number of subframes for different detectors.